# Finding Alvarez (and Others) in the A-League

SkillCorner X PySport Analytics Cup 2026: Research Track

A player similarity study using broadcast tracking data to identify A-League players matching world-class archetypes across forwards, defenders, and goalkeepers.

## 1. Introduction

In Europe, there are many excellent calibre players who could change the course of the game. One of them is Julián Alvarez who represents a modern forward archetype characterised by intelligent movement, spatial awareness, and clinical finishing. This research asks whether SkillCorner tracking data can identify A-League players with similar characteristics, primarly for Julián Alvarez then extended to other profiles with players who had good representation in online free datasets such as StatsBomb.

In this notebook, I focused on Julián Alvarez case, while in the related streamlit app, I have implemented richer interface with scouting profiles for other attackers, defenders and goalkeepers ([run using run.sh](run.sh)), ofcourse you can change here in the code to other profiles as well. Using data from 10 A-League matches, I construct player profiles based on final third entry patterns and compute similarity scores against archetypes programmatically derived from StatsBomb event data. The system supports 12 player archetypes across forwards, defenders, and goalkeepers; all computed from World Cup 2022 statistics.

The goal is to demonstrate how to scout potential players from data-driven archetypes by augmenting different data sources.

### Technical Approach - Data Flow Diagram

![Data Flow Diagram](assets/data_flow.png)

*StatsBomb World Cup 2022 data provides archetype targets; SkillCorner A-League data builds player profiles; Similarity Engine ranks matches.*

### Archetypes by Position

![Archetypes by Position](assets/archetypes_sankey.png)

*12 archetypes total: 6 Forwards, 3 Defenders, 3 Goalkeepers → 12 position-specific rankings.*

## 2. Methods

### 2.1 The Alvarez Archetype

The Alvarez archetype is computed from StatsBomb free event data using the `src/statsbomb/` package. From World Cup 2022 matches, the system extracts key statistics:

| Metric | Actual Value | Interpretation |
|--------|--------------|----------------|
| Shots | 11 | Active in final third |
| Goals | 4 | Clinical finisher |
| Conversion Rate | 36.4% | Elite finishing (95th percentile) |
| Shot Accuracy | 72.7% | Highly accurate |
| Dribble Success | 0.0% | Creates through movement, not dribbling |

The low dribble success is critical: Alvarez creates danger through intelligent movement and positioning, not individual dribbling. This maps to a low `carry_pct` target in the similarity scoring.

```python
from src.core.archetype import Archetype
archetype = Archetype.from_statsbomb("alvarez")
```

The system supports 12 other archetypes across forwards, defenders, and goalkeepers. Run `streamlit run app.py` for interactive comparison.

### 2.2 ML-Calibrated Feature Weights

I trained GradientBoosting classifiers for each position to empirically determine feature importance:

| Position | Target Variable | AUC | Events |
|----------|-----------------|-----|--------|
| Forwards | lead_to_shot | 0.656 | 245 entries |
| Defenders | stop_possession_danger | 0.845 | 8,911 engagements |
| Goalkeepers | pass_success | 0.993 | 497 distributions |

**Forward weights** (from entry danger prediction):

| Feature | ML Importance | Final Weight |
|---------|---------------|--------------|
| avg_separation | 16.2% | 23% |
| danger_rate | (target) | 18% |
| avg_entry_speed | 11.8% | 17% |
| avg_defensive_line_dist | 10.3% | 15% |
| central_pct | 8.5% | 12% |

**Defender weights** (from stop danger prediction): engagement distance (17%), pressing rate (4%), goal-side positioning (3%), with outcome metrics (stop/reduce danger, beaten rates) weighted by importance.

**Goalkeeper weights** (from pass success prediction): pass distance dominates (98.6% ML importance), balanced with style differentiators (long/short pass preference) for archetype comparison.

### 2.3 Similarity Scoring

I compute weighted cosine similarity on z-score normalised features. The target profile uses percentile-mapped values from StatsBomb statistics. For attackers, players with fewer than three entries are excluded to ensure profile reliability.

Full methodology details: [docs/methodology.md](docs/methodology.md) | Metric definitions: [docs/glossary.md](docs/glossary.md)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from src.data.loader import load_all_events, add_team_names
from src.analysis.entries import detect_entries, classify_entries
from src.analysis.profiles import build_player_profiles, filter_profiles
from src.core.similarity import SimilarityEngine
from src.analysis.defenders import detect_defensive_actions, build_defender_profiles, filter_defender_profiles
from src.analysis.goalkeepers import detect_gk_actions, build_goalkeeper_profiles, filter_goalkeeper_profiles
from src.core.archetype import Archetype

# Load archetype programmatically from StatsBomb World Cup 2022 data
# Available archetypes (12 total):
#   Forwards:    "alvarez", "giroud", "kane", "lewandowski", "rashford", "en_nesyri"
#   Defenders:   "gvardiol", "romero", "hakimi"
#   Goalkeepers: "lloris", "livakovic", "bounou"
#
# Change this line to use a different archetype for the entire analysis:
ARCHETYPE_KEY = "alvarez"

# Position mapping
FORWARD_KEYS = ["alvarez", "giroud", "kane", "lewandowski", "rashford", "en_nesyri"]
DEFENDER_KEYS = ["gvardiol", "romero", "hakimi"]
GOALKEEPER_KEYS = ["lloris", "livakovic", "bounou"]

# Defender weights (ML-calibrated, AUC 0.845)
DEFENDER_WEIGHTS = {
    "stop_danger_rate": 0.30, "avg_engagement_distance": 0.17, "reduce_danger_rate": 0.15,
    "beaten_by_possession_rate": 0.15, "beaten_by_movement_rate": 0.08, "force_backward_rate": 0.08,
    "pressing_rate": 0.04, "goal_side_rate": 0.03,
}

# Goalkeeper weights (ML-calibrated, AUC 0.993)
GOALKEEPER_WEIGHTS = {
    "pass_success_rate": 0.25, "avg_pass_distance": 0.20, "long_pass_pct": 0.15,
    "short_pass_pct": 0.10, "high_pass_pct": 0.10, "quick_distribution_pct": 0.10,
    "under_pressure_pct": 0.05, "to_attacking_third_pct": 0.05,
}

# Target profiles for radar charts (normalized 0-100 scale)
# Defender target profiles based on playing style
DEFENDER_TARGET_PROFILES = {
    "gvardiol": {  # Ball-playing CB - composed, positional
        "stop_danger_rate": 75, "reduce_danger_rate": 70, "force_backward_rate": 65,
        "pressing_rate": 50, "goal_side_rate": 85, "avg_engagement_distance": 60,
    },
    "romero": {  # Aggressive CB - front-foot defending
        "stop_danger_rate": 80, "reduce_danger_rate": 75, "force_backward_rate": 80,
        "pressing_rate": 75, "goal_side_rate": 70, "avg_engagement_distance": 55,
    },
    "hakimi": {  # Attacking wing-back - forward positioning
        "stop_danger_rate": 55, "reduce_danger_rate": 60, "force_backward_rate": 50,
        "pressing_rate": 70, "goal_side_rate": 50, "avg_engagement_distance": 70,
    },
}

# Goalkeeper target profiles based on distribution style
GOALKEEPER_TARGET_PROFILES = {
    "lloris": {  # Sweeper-keeper - quick, long distribution
        "pass_success_rate": 75, "avg_pass_distance": 80, "long_pass_pct": 70,
        "short_pass_pct": 30, "high_pass_pct": 60, "quick_distribution_pct": 75,
    },
    "livakovic": {  # Shot-stopper - safer distribution
        "pass_success_rate": 85, "avg_pass_distance": 50, "long_pass_pct": 40,
        "short_pass_pct": 60, "high_pass_pct": 45, "quick_distribution_pct": 50,
    },
    "bounou": {  # Modern GK - balanced approach
        "pass_success_rate": 80, "avg_pass_distance": 65, "long_pass_pct": 55,
        "short_pass_pct": 45, "high_pass_pct": 50, "quick_distribution_pct": 60,
    },
}

# Determine position and load appropriate archetype
if ARCHETYPE_KEY in FORWARD_KEYS:
    position_type = "forward"
    archetype = Archetype.from_statsbomb(ARCHETYPE_KEY)
    radar_target_profile = None  # Forward archetypes use archetype.target_profile
elif ARCHETYPE_KEY in DEFENDER_KEYS:
    position_type = "defender"
    archetype = Archetype.from_statsbomb(ARCHETYPE_KEY)
    archetype.weights = DEFENDER_WEIGHTS  # Override with defender weights
    radar_target_profile = DEFENDER_TARGET_PROFILES.get(ARCHETYPE_KEY, {})
else:
    position_type = "goalkeeper"
    archetype = Archetype.from_statsbomb(ARCHETYPE_KEY)
    archetype.weights = GOALKEEPER_WEIGHTS  # Override with goalkeeper weights
    radar_target_profile = GOALKEEPER_TARGET_PROFILES.get(ARCHETYPE_KEY, {})

print(f"Using archetype: {archetype.name} (Position: {position_type})")

In [ ]:
# Load data based on position type
events = load_all_events()
events = add_team_names(events)

if position_type == "forward":
    entries = detect_entries(events)
    entries = classify_entries(entries)
    profiles = build_player_profiles(entries)
    profiles = filter_profiles(profiles, min_entries=3)
    n_events = len(entries)
    event_name = "entries"
elif position_type == "defender":
    actions = detect_defensive_actions(events)
    profiles = build_defender_profiles(actions)
    profiles = filter_defender_profiles(profiles, min_engagements=5)
    n_events = len(actions)
    event_name = "engagements"
else:  # goalkeeper
    actions = detect_gk_actions(events)
    profiles = build_goalkeeper_profiles(actions)
    profiles = filter_goalkeeper_profiles(profiles, min_distributions=10)
    n_events = len(actions)
    event_name = "distributions"

print(f"Loaded {n_events} {event_name} from {events['match_id'].n_unique()} matches")
print(f"Built profiles for {len(profiles)} players")

In [ ]:
# Compute similarity using the selected archetype
engine = SimilarityEngine(archetype)
engine.fit(profiles)
ranked = engine.rank(top_n=10)

In [ ]:
from src.analysis.danger_model import train_danger_model, train_defender_model, train_goalkeeper_model

# Train ML model based on position type
if position_type == "forward":
    ml_result = train_danger_model(entries)
    print(f"Forward ML Model CV AUC: {ml_result['cv_auc_mean']:.3f} ± {ml_result['cv_auc_std']:.3f}")
elif position_type == "defender":
    ml_result = train_defender_model(actions)
    print(f"Defender ML Model CV AUC: {ml_result['cv_auc_mean']:.3f} ± {ml_result['cv_auc_std']:.3f}")
else:  # goalkeeper
    ml_result = train_goalkeeper_model(actions)
    print(f"Goalkeeper ML Model CV AUC: {ml_result['cv_auc_mean']:.3f} ± {ml_result['cv_auc_std']:.3f}")

## 3. Results and Discussions

### 3.1 Results
The analysis identified 31 players with three or more final third entries across the 10 matches. Figure 1 shows the top 10 players ranked by similarity to the Alvarez archetype (computed from StatsBomb World Cup 2022 data: 36.4% conversion, 0% dribble reliance).

In [ ]:
from src.visualization.radar import plot_similarity_ranking
import matplotlib.pyplot as plt

fig = plot_similarity_ranking(ranked, top_n=10, title="Figure 1: Similarity Rankings")
plt.show()

In [ ]:
# Table 1: Top 5 candidates with ages included
from src.data.player_ages import add_ages_to_profiles

ranked_with_ages = add_ages_to_profiles(ranked)

# Position-specific display columns
if position_type == "forward":
    display_cols = ["rank", "player_name", "age", "team_name", "similarity_score", 
                    "total_entries", "danger_rate", "central_pct", "avg_separation"]
elif position_type == "defender":
    display_cols = ["rank", "player_name", "age", "team_name", "similarity_score",
                    "total_engagements", "stop_danger_rate", "pressing_rate", "goal_side_rate"]
else:  # goalkeeper
    display_cols = ["rank", "player_name", "age", "team_name", "similarity_score",
                    "total_distributions", "pass_success_rate", "avg_pass_distance", "long_pass_pct"]

available = [c for c in display_cols if c in ranked_with_ages.columns]
ranked_with_ages.head(5).select(available)

Figure 2 compares the profiles of the top three candidates across the key features used in the similarity calculation in a radar stype.

In [ ]:
from src.visualization.radar import plot_radar_comparison, normalize_for_radar

# Position-specific features for radar chart
if position_type == "forward":
    features = ["avg_separation", "central_pct", "half_space_pct", "danger_rate", 
                "avg_entry_speed", "avg_passing_options", "carry_pct"]
elif position_type == "defender":
    features = ["stop_danger_rate", "reduce_danger_rate", "force_backward_rate",
                "pressing_rate", "goal_side_rate", "avg_engagement_distance"]
else:  # goalkeeper
    features = ["pass_success_rate", "avg_pass_distance", "long_pass_pct",
                "short_pass_pct", "high_pass_pct", "quick_distribution_pct"]

# Filter to only features that exist in the data
features = [f for f in features if f in ranked.columns]

if len(features) < 3:
    print(f"Warning: Only {len(features)} features available for radar chart")
else:
    norm_profiles = normalize_for_radar(ranked, features)

    top_3 = []
    for row in norm_profiles.head(3).to_dicts():
        player_data = {"name": f"{row['player_name']} ({row.get('team_name', '')})"}
        for f in features:
            player_data[f] = row.get(f"{f}_norm", 0)
        top_3.append(player_data)

    # Use position-specific target profile (defined in Cell 4)
    # For forwards: use archetype.target_profile
    # For defenders/goalkeepers: use radar_target_profile (position-specific)
    if radar_target_profile:
        target_profile = {f: radar_target_profile.get(f, 50) for f in features}
    else:
        target_profile = {f: archetype.target_profile.get(f, 50) for f in features}
    
    archetype_display_name = archetype.name.replace("_", " ").title()

    fig = plot_radar_comparison(
        top_3, 
        features=features, 
        title=f"Figure 2: {position_type.title()} Profile Comparison",
        include_alvarez=True,
        target_profile=target_profile,
        target_name=archetype_display_name
    )
    plt.show()

### 3.2. Discussion

The radar chart reveals interesting patterns when comparing A-League candidates against the Alvarez target profile (shown as the dashed orange line). The target profile emphasises high separation, danger rate, and central positioning while deliberately setting a low carry percentage, reflecting that Alvarez creates through movement rather than dribbling.

**T. Imai** from Western United emerges as the top match with exceptional separation (5.64m) and a 40% danger rate. His high separation value indicates he consistently finds space away from defenders, a key characteristic of the Alvarez archetype. The combination of space creation and shot generation mirrors Alvarez's intelligent movement patterns.

**N. Atkinson** from Melbourne City follows with balanced metrics: 16.7% danger rate, 16.7% central entry percentage, and 4.41m separation. His profile suggests a player comfortable operating in central areas while maintaining good spacing from defenders.

**K. Bos** from Melbourne Victory presents similar characteristics with 4.14m separation and 16.7% danger rate. His profile indicates consistent space-finding ability, though with entries predominantly from wide areas (0% central).

**Z. Clough** from Adelaide United, ranked fourth, shows the highest danger rate in the top five at 50% and the highest central percentage at 25%. His separation value (3.41m) suggests he operates slightly closer to defenders but compensates with clinical output when receiving the ball.

The current rankings heavily weight separation (avg_separation at 23%), which identifies players who find space between defensive lines. This aligns with Alvarez's playing style of intelligent off-ball movement rather than relying on pace or dribbling to beat defenders.

### 3.3 Limitations

Several limitations should be acknowledged:

1. Sample size: 10 matches constrains reliability, with some players having as few as three entries
2. Cross-dataset mapping: StatsBomb events → SkillCorner tracking requires conceptual translation
3. Context differences: A-League tactical context differs from World Cup matches where archetypes are derived
4. Entry method homogeneity: 97.6% of entries are classified as carries, leaving insufficient variation to distinguish players by entry style

Despite these constraints, the methodology demonstrates that broadcast tracking data can identify players matching specific stylistic archetypes. This has practical applications for recruitment, where clubs might seek players fitting particular tactical profiles.

## 4. Conclusion

This research demonstrates a methodology for player similarity analysis using SkillCorner broadcast tracking data. By computing archetypes programmatically from StatsBomb event data and applying ML-calibrated similarity weights across all positions, I identified A-League players who exhibit similar characteristics to world-class archetypes.

### Key Contributions

1. Archetype generation: 12 player archetypes computed from StatsBomb World Cup 2022 data
2. ML-calibrated weights for all positions: GradientBoosting models trained per position type:
   - Forwards: AUC 0.656 (predict shot creation)
   - Defenders: AUC 0.845 (predict stopping danger)
   - Goalkeepers: AUC 0.993 (predict distribution success)
3. Cross-domain mapping: StatsBomb events → SkillCorner tracking target profiles
4. Extensible framework: Easy to add new archetypes or apply to other SkillCorner datasets

### Interactive Tool

For interactive comparison across all positions:

- **Live Demo**: [huggingface.co/spaces/KarimElgammal/analytics-cup-research](https://huggingface.co/spaces/KarimElgammal/analytics-cup-research)
- **Documentation**: [karimelgammal.github.io/analytics_cup_research](https://karimelgammal.github.io/analytics_cup_research/)
- **Local**: Run `./run.sh`

Supports 12 archetypes: 6 forwards (Alvarez, Giroud, Kane, Lewandowski, Rashford, En-Nesyri), 3 defenders (Gvardiol, Romero, Hakimi), 3 goalkeepers (Lloris, Livakovic, Bounou).

For scouts and analysts, this provides a data-driven starting point for identifying players who might fit specific tactical roles.

## 5. AI-Generated Scouting Insight

The following insight is generated using LLM APIs (GitHub Models or HuggingFace), synthesising the analysis above into a scouting recommendation.

**No token? or running me on Binder** Try the [Live Demo on HuggingFace](https://huggingface.co/spaces/KarimElgammal/analytics-cup-research) for AI insights without setup.

In [ ]:
# AI Model Configuration
# Choose your backend and model:

# BACKEND OPTIONS:
#   "github"      - GitHub Models (requires github_token.txt)
#   "huggingface" - HuggingFace Inference (requires hf_token.txt)

# GITHUB MODELS:
#   "phi-4"       - Microsoft Phi-4 (recommended, fast)
#   "gpt-4o-mini" - OpenAI GPT-4o Mini
#   "grok-3-mini" - xAI Grok 3 Mini (expensive, reasoning model)

# HUGGINGFACE MODELS:
#   "llama-3.1-8b" - Meta Llama 3.1 8B (recommended)
#   "llama-3.2-3b" - Meta Llama 3.2 3B (faster)
#   "qwen-2.5-7b"  - Qwen 2.5 7B
#   "smollm3-3b"   - SmolLM3 3B
#   "gemma-2-2b"   - Google Gemma 2 2B

SELECTED_MODEL = 'smollm3-3b' #"phi-4"  # Change this to use a different model

# Token setup (uncomment one):
# set_github_token("your_github_token_here")
# set_hf_token("your_hf_token_here")

from IPython.display import display, Markdown
from src.utils.ai_insights import (
    generate_similarity_insight, 
    has_valid_token, 
    set_github_token,
    set_hf_token,
    get_model_backend,
    MODEL_DISPLAY_NAMES,
)

if has_valid_token():
    backend = get_model_backend(SELECTED_MODEL)
    model_name = MODEL_DISPLAY_NAMES.get(SELECTED_MODEL, SELECTED_MODEL)
    print(f"Using: {model_name} (Backend: {backend})")
    
    insight = generate_similarity_insight(
        ranked, 
        archetype, 
        top_n=5, 
        position_type=position_type,
        model=SELECTED_MODEL,
    )
    display(Markdown(insight))
else:
    print("No token found.")
    print("Options:")
    print("  1. Add token to github_token.txt or hf_token.txt")
    print("  2. Use set_github_token() or set_hf_token() above")
    print("  3. Try the Live Demo: https://huggingface.co/spaces/KarimElgammal/analytics-cup-research")